## Encoder Formular

### x = self.norm1(x + self._sa_block(x, src_mask, src_key_padding_mask))
### x = self.norm2(x + self._ff_block(x))

In [1]:
import torch
import torch.nn as nn
import math
torch.manual_seed(1773) 



### Embedding and Positional Encoding and Setting the Weights

In [2]:
E = 2
n_head = 1
embedding = torch.tensor([[1.1,-0.5],[0.6,0.9],[0.8,0.6]])
positional_embedding = torch.tensor([[0,1],[0.84,0.54],[0.90,-0.41]])
query = (embedding+ positional_embedding).reshape((1,3,2))
encoder_layer = nn.TransformerEncoderLayer(d_model=E, nhead=1,batch_first=True,dropout=0,dim_feedforward=2)
zeros_tensor = torch.nn.Parameter(torch.zeros(2))
encoder_layer.linear1.bias = zeros_tensor
encoder_layer.linear2.bias = zeros_tensor
out = encoder_layer(query)

### Self Attention 

In [3]:
query = torch.tensor([[[0.7204, 0.0731],
          [0.9699, 0.1078],
          [0.8829, 0.4132]]])


key = query
value = query
weight = encoder_layer.state_dict()['self_attn.in_proj_weight']

q,k, v =  query.matmul(weight.t()).chunk(3, dim=-1)
v = v.reshape(3,2)
q = q.reshape(3,2)
k = k.reshape(3,2)
q = q / math.sqrt(E)
attn1 = torch.matmul(q,k.T)
m = nn.Softmax(dim=-1)
attn = m(attn1)
output = torch.matmul(attn, v)
output = output.reshape((3,2))
out_proj = encoder_layer.state_dict()['self_attn.out_proj.weight']
output_last = output.matmul(out_proj.t())
x = key.reshape((3,2)) + output_last

### Normalization

In [4]:
copied_data = x.clone()
layer_norm1 = nn.LayerNorm(2)
x = layer_norm1(x)

## Fully Connected Layer

In [5]:
b = x.squeeze()
linear1 = torch.matmul(b,encoder_layer.state_dict()['linear1.weight'].T)
m = nn.ReLU()
linear1_relu = m(linear1)
linear2 = torch.matmul(linear1_relu,encoder_layer.state_dict()['linear2.weight'].T)
x = x + linear2

## Normalization Two

In [11]:
layer_norm2 = nn.LayerNorm(2)
x = layer_norm2(x)